# Imports

In [ ]:
import numpy as np
import datetime, os
from tensorflow.keras.utils import get_file
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import datetime

!wget https://github.com/camsovangauthier/coursesNN/raw/main/Fashion_MNIST_5classes_augmentation.h5
!wget https://github.com/gaudel/NN/raw/main/pretrained_model.h5

# Usefull functions

In [ ]:
def build_CNN(input_dim, output_dim, lr=0.001):
    model = Sequential()

    #Reduire les dim, apprendre les poids
    model.add(Conv2D(32, (3, 3), input_shape = input_dim, activation="relu")) 
    #Q1: Compléter ici
    model.add(MaxPooling2D(pool_size=(2, 2))) # Représentation image plus réduite
    model.add(Dropout(rate = 0.1))   # Mais le but est bien d'unifier l'apprentissage
  
    model.add(Conv2D(64, (3, 3), activation="relu")) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    model.add(Dropout(0.1))
  
    model.add(Conv2D(64, (3, 3), activation="relu")) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    model.add(Dropout(0.1))
  
    model.add(Flatten()) # Flatten() conversion de matrices 3D à un vecteur 1D
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(output_dim, activation = "softmax"))

    #Q1: Compilation du modèle
    model.compile(loss = "categorical_crossentropy", optimizer= "rmsprop", metrics=['accuracy'])
    return model

In [ ]:
def build_CNN_sans_Dropout(input_dim, output_dim, lr=0.001):
    model = Sequential()

    #Reduire les dim, apprendre les poids
    model.add(Conv2D(32, (3, 3), input_shape = input_dim, activation="relu")) 
    #Q1: Compléter ici
    model.add(MaxPooling2D(pool_size=(2, 2))) # Représentation image plus réduite
  
    model.add(Conv2D(64, (3, 3), activation="relu")) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
  
    model.add(Conv2D(64, (3, 3), activation="relu")) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
  
    model.add(Flatten()) # Flatten() conversion de matrices 3D à un vecteur 1D
    model.add(Dense(64, activation='relu'))

    model.add(Dense(output_dim, activation = "softmax"))

    #Q1: Compilation du modèle
    model.compile(loss = "categorical_crossentropy", optimizer= "rmsprop", metrics=['accuracy'])
    return model

In [ ]:
def build_CNN_from_pretrained(file_name, output_dim, lr=0.001):
  # Q4: compléter l'appel à la fonction load_model()
  model = load_model( filepath = file_name)
  # Q4: retirer et ajouter la/les couches nécessaires ici
  model.pop()
  model.add(Dense(output_dim, activation = "softmax"))
  # Q4: Compilation du modèle
  return model

# Main

## Hyper-parameters

In [ ]:
epochs = 50
batch_size = 64

## Initialization

In [ ]:
rng = np.random.default_rng()

## Data

### Download data

In [ ]:
data_f = get_file('data_fashion.zip', 'https://github.com/camsovangauthier/coursesNN/raw/main/data_fashion.zip', extract=True)

def foo(data_src):
  if type(data_src) == str:
    images, labels = next(ImageDataGenerator(rescale=1./255).flow_from_directory(data_f[:-4] + '/' + data_src, batch_size=25))
  else:
    images, labels = next(data_src)

  print(f'shape of features: {images.shape[1:]}')
  print(f'shape of labels: {labels.shape[1:]}')
  class_names = ['Bag', 'Dress']
  plt.figure(figsize=(10,10))
  for i in range(25):
      plt.subplot(5,5,i+1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      plt.imshow(images[i], cmap=plt.cm.binary)
      plt.xlabel(class_names[np.argmax(labels[i])] + f' ({labels[i]})')
  plt.show()

print('--- train data ---')
foo('train')

print('--- validation data ---')
foo('valid')

### Data generators

In [ ]:
#Q2: Ajouter l'attribut rescale aux générateurs
train_generator = ImageDataGenerator(rescale = 1./255 )
    # rescale : réduire ou augmenter les données
valid_generator = ImageDataGenerator(rescale = 1./255 )

train_flow = train_generator.flow_from_directory(data_f[:-4]+'/train', target_size=(28, 28), batch_size = batch_size)
valid_flow = valid_generator.flow_from_directory(data_f[:-4]+'/valid', target_size=(28, 28), batch_size = batch_size)

print('--- train data ---')
foo(train_flow)

print('--- validation data ---')
foo(valid_flow)

In [ ]:
cnn_model = build_CNN( input_dim = (28, 28, 3), output_dim = 2, lr=0.001)
cnn_model.summary()

In [ ]:

cnn_model = build_CNN( input_dim = (28, 28, 3), output_dim = 2, lr=0.001)

model = cnn_model.fit(train_flow, batch_size = batch_size, epochs = 20, validation_data = valid_flow)

cnn_model.evaluate(valid_flow, verbose = 2)

In [ ]:
cnn_model.evaluate(valid_flow, verbose = 2)

In [ ]:
loss = model.history['loss']
val_loss = model.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Estimation sans Dropout
cnn_model = build_CNN_sans_Dropout( input_dim = (28, 28, 3), output_dim = 2, lr=0.001)

model = cnn_model.fit(train_flow, batch_size = batch_size, epochs = 20, validation_data = valid_flow)

cnn_model.evaluate(valid_flow, verbose = 2)

In [ ]:
loss = model.history['loss']
val_loss = model.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Q3: Augmentation des données
train_generator = ImageDataGenerator(rescale = 1./255, featurewise_center = True, featurewise_std_normalization = True, 
                                     rotation_range = 40, zoom_range = 0.2, shear_range = 0.2, horizontal_flip = True)
    # rescale : réduire ou augmenter les données
valid_generator = ImageDataGenerator(rescale = 1./255, featurewise_center = True, featurewise_std_normalization = True, 
                                     rotation_range = 40, zoom_range = 0.2, shear_range = 0.2, horizontal_flip = True)

train_flow = train_generator.flow_from_directory(data_f[:-4]+'/train', target_size=(28, 28), batch_size = batch_size)
valid_flow = valid_generator.flow_from_directory(data_f[:-4]+'/valid', target_size=(28, 28), batch_size = batch_size)

print('--- train data ---')
foo(train_flow)

print('--- validation data ---')
foo(valid_flow)

# target_size : taille finale des images après l'augmentation des données
# input_dim = (28, 28, 3): taille des données d'entrée du réseau


In [ ]:
cnn_model = build_CNN( input_dim = (28, 28, 3), output_dim = 2, lr=0.001)

model = cnn_model.fit(train_flow, batch_size = batch_size, epochs = 20, validation_data = valid_flow)

cnn_model.evaluate(valid_flow, verbose = 2)

In [ ]:
loss = model.history['loss']
val_loss = model.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
 cnn_model_5_classes = build_CNN_from_pretrained("Fashion_MNIST_5classes_augmentation.h5", 2)

 cnn_model_5_classes.summary()




In [ ]:
 cnn_model_5_classes = build_CNN_from_pretrained("Fashion_MNIST_5classes_augmentation.h5", 2)

 cnn_model_5_classes.summary()

In [ ]:
 cnn_model.summary()


In [ ]:
#Q4: Estimation des paramètres sur nos données avec le modèle pré-entrainé
train_generator = ImageDataGenerator(rescale = 1./255, featurewise_center = True, featurewise_std_normalization = True, 
                                     rotation_range = 40, zoom_range = 0.2, shear_range = 0.2, horizontal_flip = True)
    # rescale : réduire ou augmenter les données
valid_generator = ImageDataGenerator(rescale = 1./255, featurewise_center = True, featurewise_std_normalization = True, 
                                     rotation_range = 40, zoom_range = 0.2, shear_range = 0.2, horizontal_flip = True)

train_flow = train_generator.flow_from_directory(data_f[:-4]+'/train', target_size=(32, 32), batch_size = batch_size)
valid_flow = valid_generator.flow_from_directory(data_f[:-4]+'/valid', target_size=(32, 32), batch_size = batch_size)

print('--- train data ---')
foo(train_flow)

print('--- validation data ---')
foo(valid_flow)

In [ ]:
#Q4: Estimation des paramètres sur nos données avec le modèle pré-entrainé à 5 classes
cnn_model = build_CNN_from_pretrained("Fashion_MNIST_5classes_augmentation.h5", 2)

model = cnn_model.fit(train_flow, batch_size = batch_size, epochs = 20, validation_data = valid_flow)

cnn_model.evaluate(valid_flow, verbose = 2)

In [ ]:
loss = model.history['loss']
val_loss = model.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Q5: Estimation des paramètres sur nos données avec le modèle pré-entrainé à 10 classes
train_generator = ImageDataGenerator(rescale = 1./255, featurewise_center = True, featurewise_std_normalization = True, 
                                     rotation_range = 40, zoom_range = 0.2, shear_range = 0.2, horizontal_flip = True)
    # rescale : réduire ou augmenter les données
valid_generator = ImageDataGenerator(rescale = 1./255, featurewise_center = True, featurewise_std_normalization = True, 
                                     rotation_range = 40, zoom_range = 0.2, shear_range = 0.2, horizontal_flip = True)

train_flow = train_generator.flow_from_directory(data_f[:-4]+'/train', target_size=(32, 32), batch_size = batch_size)
valid_flow = valid_generator.flow_from_directory(data_f[:-4]+'/valid', target_size=(32, 32), batch_size = batch_size)

print('--- train data ---')
foo(train_flow)

print('--- validation data ---')
foo(valid_flow)

In [ ]:
#Q5: Estimation des paramètres sur nos données avec le modèle pré-entrainé à 10 classes
cnn_model = build_CNN_from_pretrained("pretrained_model.h5", 2)

model = cnn_model.fit(train_flow, batch_size = batch_size, epochs = 20, validation_data = valid_flow)

cnn_model.evaluate(valid_flow, verbose = 2)

In [ ]:
cnn_model.summary()

In [ ]:
loss = model.history['loss']
val_loss = model.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Build

In [ ]:
# from_scratch = False
from_scratch = True # au début
if from_scratch:
  # Q2: compléter l'appel à la fonction build_CNN()
  cnn_model = build_CNN( input_dim = (28, 28, 3), output_dim = 2, lr=0.001)
  label = 'from_scratch'
else:
  cnn_model = build_CNN_from_pretrained("Fashion_MNIST_5classes_augmentation.h5", 2)
  label = 'from_pretrained'
cnn_model.summary()

## Fit & evaluate

In [ ]:
#Q2: compléter l'appel aux fonctions fit() et evaluate()
tensorboard_callback = TensorBoard(log_dir='logs/' + label + '__' + datetime.datetime.now().strftime("%d-%m_%Hh%M"), histogram_freq=int(epochs/10))

cnn_model.fit(train_flow, batch_size = batch_size, epochs = epochs, validation_data = valid_flow)

cnn_model.evaluate(valid_flow, verbose=2)


# Tensorboard

## Initialization

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Start tensorboard

In [ ]:
%tensorboard --logdir logs